# Lego Graph (TigerGraph and Kaggle)

## Create your TigerGraph Graph

### Graph

In [ ]:
!pip install pyTigerGraph

In [ ]:
import pyTigerGraph as tg 

conn = tg.TigerGraphConnection(host="https://test.i.tgcloud.io/", password="tigergraph")

In [ ]:
# NOTE: ONLY UNCOMMENT FOR EMERGENCIES!! IT WILL DELETE EVERYTHING FROM YOUR GRAPH!

# print(conn.gsql('''DROP ALL'''))

Dropping all, about 1 minute ...
Abort all active loading jobs
Try to abort all loading jobs on graph LegoGraph, it may take a while ...
[ABORT_SUCCESS] No active Loading Job to abort.
Resetting GPE...
Successfully reset GPE
Stopping GPE GSE
Successfully stopped GPE GSE in 7.410 seconds
Clearing graph store...
Successfully cleared graph store
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.094 seconds
Everything is dropped.


In [ ]:
print(conn.gsql('''
CREATE VERTEX Colour(PRIMARY_ID colour_id STRING, colour_name STRING, rgb STRING, transparent STRING)
CREATE VERTEX Inventory(PRIMARY_ID inventory_id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Version(PRIMARY_ID version_id INT) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Lego_Set(PRIMARY_ID set_num STRING, num_parts INT) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Part(PRIMARY_ID part_num STRING, part_name STRING)
CREATE VERTEX Part_Category(PRIMARY_ID part_cat_num STRING, part_name STRING)
CREATE VERTEX Theme(PRIMARY_ID theme_id STRING, theme_name STRING, theme_number INT) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Year(PRIMARY_ID year INT) WITH PRIMARY_ID_AS_ATTRIBUTE="true"

CREATE UNDIRECTED EDGE INVENTORY_VERSION(FROM Inventory, TO Version)
CREATE UNDIRECTED EDGE INVENTORY_LEGO_SET(FROM Inventory, TO Lego_Set, quantity INT)
CREATE UNDIRECTED EDGE LEGO_SET_THEME(FROM Lego_Set, TO Theme)
CREATE UNDIRECTED EDGE LEGO_SET_YEAR(FROM Lego_Set, TO Year)
CREATE UNDIRECTED EDGE INVENTORY_PART(FROM Inventory, TO Part, quantity INT, spare STRING)
CREATE UNDIRECTED EDGE PART_COLOUR(FROM Part, TO Colour)
CREATE UNDIRECTED EDGE PART_CATEGORY_PART(FROM Part_Category, TO Part)
CREATE DIRECTED EDGE PARENT_THEME(FROM Theme, TO Theme)
'''))

The vertex type Colour is created.
The vertex type Inventory is created.
The vertex type Version is created.
The vertex type Lego_Set is created.
The vertex type Part is created.
The vertex type Part_Category is created.
The vertex type Theme is created.
The vertex type Year is created.
The edge type INVENTORY_VERSION is created.
The edge type INVENTORY_LEGO_SET is created.
The edge type LEGO_SET_THEME is created.
The edge type LEGO_SET_YEAR is created.
The edge type INVENTORY_PART is created.
The edge type PART_COLOUR is created.
The edge type PART_CATEGORY_PART is created.
The edge type PARENT_THEME is created.


In [ ]:
print(conn.gsql('''
CREATE GRAPH LegoGraph(
  Colour, Inventory, Version, Lego_Set, Part, Part_Category, Theme, Year,
  INVENTORY_VERSION, INVENTORY_LEGO_SET, LEGO_SET_THEME, LEGO_SET_YEAR, INVENTORY_PART, PART_COLOUR, PART_CATEGORY_PART, PARENT_THEME
)
'''))

Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 0.333 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.097 seconds
The graph LegoGraph is created.


In [ ]:
conn.graphname = "LegoGraph"
conn.apiToken = conn.getToken(conn.createSecret())

### Grab Kaggle Data





In [ ]:
! pip install -q kaggle

Go to your Settings, click Account, scroll down to API. 

**Press "Expire" then "Create New Token". It will download a kaggle.json; upload it to the selector below.**



In [ ]:
from google.colab import files

files.upload()

Saving kaggle (2).json to kaggle (2).json


{'kaggle (2).json': b'{"username":"genericp3rson","key":"c6f4d01eccb13b0ae48a5ed10e7764c9"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                           title                                              size  lastUpdated          downloadCount  
------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                   Reddit Vaccine Myths                              233KB  2021-07-07 19:18:09           8905  
crowww/a-large-scale-fish-dataset                             A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           5460  
imsparsh/musicnet-dataset                                     MusicNet Dataset                                   22GB  2021-02-18 14:12:19           1962  
dhruvildave/wikibooks-dataset                                 Wikibooks Dataset                                   2GB  2021-07-03 18:37:20           2365  
prom

In [ ]:
! kaggle datasets download -d rtatman/lego-database
! unzip lego-database.zip -d train
! ls train

  0% 0.00/2.94M [00:00<?, ?B/s]
100% 2.94M/2.94M [00:00<00:00, 96.8MB/s]
Archive:  lego-database.zip
  inflating: train/colors.csv        
  inflating: train/downloads_schema.png  
  inflating: train/inventories.csv   
  inflating: train/inventory_parts.csv  
  inflating: train/inventory_sets.csv  
  inflating: train/part_categories.csv  
  inflating: train/parts.csv         
  inflating: train/sets.csv          
  inflating: train/themes.csv        
colors.csv	      inventory_parts.csv  parts.csv
downloads_schema.png  inventory_sets.csv   sets.csv
inventories.csv       part_categories.csv  themes.csv


### Loading Data

In [ ]:
import pandas as pd 

colours = pd.read_csv("train/colors.csv")
parts = pd.read_csv("train/parts.csv")
lego_sets = pd.read_csv("train/sets.csv")
themes = pd.read_csv("train/themes.csv")
inventories = pd.read_csv("train/inventories.csv")
inventory_parts = pd.read_csv("train/inventory_parts.csv")
inventory_sets = pd.read_csv("train/inventory_sets.csv")
part_categories = pd.read_csv("train/part_categories.csv")

In [ ]:
conn.upsertVertexDataFrame(colours, "Colour", "id", attributes={"colour_name": "name", "rgb": "rgb", "transparent": "is_trans"})
conn.upsertVertexDataFrame(parts, "Part", "part_num", attributes={"part_name": "name"})
conn.upsertVertexDataFrame(lego_sets, "Lego_Set", "set_num", attributes={"set_num": "set_num", "num_parts": "num_parts"})
conn.upsertVertexDataFrame(themes, "Theme", "id", attributes={"theme_number": "id", "theme_name": "name"})
conn.upsertVertexDataFrame(lego_sets, "Year", "year", attributes={"year": "year"})
conn.upsertVertexDataFrame(inventories, "Inventory", "id", attributes={})
conn.upsertVertexDataFrame(inventories, "Version", "version", attributes={"version_id": "version"})
conn.upsertVertexDataFrame(part_categories, "Part_Category", "id", attributes={"part_name": "name"})

conn.upsertEdgeDataFrame(inventories, "Inventory", "INVENTORY_VERSION", "Version", "id", "version", attributes={})
conn.upsertEdgeDataFrame(lego_sets, "Lego_Set", "LEGO_SET_THEME", "Theme", "set_num", "theme_id", attributes={})
conn.upsertEdgeDataFrame(parts, "Part_Category", "PART_CATEGORY_PART", "Part", from_id="part_cat_id", to_id="part_num", attributes={})
conn.upsertEdgeDataFrame(lego_sets, "Lego_Set", "LEGO_SET_YEAR", "Year", "set_num", "year", attributes={})
conn.upsertEdgeDataFrame(themes, "Theme", "PARENT_THEME", "Theme", "name", "parent_id", attributes={})
conn.upsertEdgeDataFrame(inventories, "Inventory", "INVENTORY_LEGO_SET", "Lego_Set", "id", "set_num", attributes={})
conn.upsertEdgeDataFrame(inventories, "Inventory", "INVENTORY_VERSION", "Version", "id", "version", attributes={})
conn.upsertEdgeDataFrame(inventory_parts, "Inventory", "INVENTORY_PART", "Part", "inventory_id", "part_num", attributes={"quantity": "quantity", "spare": "is_spare"})
conn.upsertEdgeDataFrame(inventory_parts, "Part", "PART_COLOUR", "Colour", "part_num", "color_id", attributes={})
conn.upsertEdgeDataFrame(inventory_sets, "Inventory", "INVENTORY_LEGO_SET", "Lego_Set", "inventory_id", "set_num", attributes={"quantity": "quantity"})

2846